# Pedestrian street networks for 21 cities
Using OpenStreetMap as a source for 
* complete road network, and 
* a pedestrian 'walk/cycle'network. 
* calculating intersection density for the pedestrian network

The process iterates over a list of city names, whose 10km buffered boundary shape files have been pre-prepared.

Carl Higgs
1 October 2018

## OSM set up

In [2]:
# Libraries used for OSM conversion
import os
import sys
import subprocess as sp
from datetime import datetime

# Libraries used for OSMnx analyses and output
import networkx as nx
import osmnx as ox
import requests
import fiona
ox.config(use_cache=True, log_console=True)
ox.__version__

from shapely.geometry import shape, MultiPolygon, Polygon

# define pedestrian network custom filter (based on OSMnx 'walk' network type, without the cycling exclusion)
pedestrian = (
             '["area"!~"yes"]' 
             '["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]'
             '["foot"!~"no"]'  
             '["service"!~"private"]' 
             '["access"!~"private"]'
             )

## Extract OSM using 10km buffered study region .poly files

The idea with this step is to extract OSM for each buffered study region, and then this file would be used to build the all non-private roads, and pedestrian roads networks.  However there is apparently a bug with the OSMnx 'graph_from_file' command; as such, OSMnx networks were constructed using the OSM version current on Overpass.de at date of processing (2/10/2018).

In [5]:
# iterate of files within root or otherwise specified directory, noting all poly files

# location of source OSM file
osm_dir = 'D:/osm/planet_archives/planet-latest_20181001.osm.pbf'

# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/study_region/'

# conversion settings
exe = 'osmconvert64-0.8.8p.exe'
exepath = 'D:/osm/'
osm_format = 'osm'

# output suffix
suffix = '_20181001'

count = 0
# Start timing the code
start_time = datetime.now()
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith(".poly"):
           # Extract OSM
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = '{root}/{filename}{suffix}.{osm_format}'.format(root = root,
                                                                         filename = filename,
                                                                         suffix = suffix,
                                                                         osm_format = osm_format)
           command = '{osmconvert} {osm} -B={poly} -o={studyregion}'.format(osmconvert = exe, 
                                                                            osm = osm_dir,
                                                                            poly = fullfile,
                                                                            studyregion = studyregion)
           sp.call(command, shell=True, cwd=exepath)
           count+=1
           print(' Extraction of .osm file for {} complete in {:.1f} minutes.'.format(filename,
                                                                                  (datetime.now() - subtime).total_seconds()/60))
            
print('\nExtracted (or attempted to extract) {} OSM portions.'.format(count))            
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))

 Extraction of .osm file for AlburyWodonga complete in 19.1 minutes.
 Extraction of .osm file for Ballarat complete in 19.2 minutes.
 Extraction of .osm file for Bendigo complete in 19.3 minutes.
 Extraction of .osm file for Cairns complete in 19.3 minutes.
 Extraction of .osm file for adelaide_gccsa_2016_10000m_0 complete in 19.5 minutes.
 Extraction of .osm file for bris_gccsa_2016_10000m_0 complete in 19.9 minutes.
 Extraction of .osm file for canberra_gccsa_2016_10000m_0 complete in 20.0 minutes.
 Extraction of .osm file for darwin_gccsa_2016_10000m_0 complete in 20.1 minutes.
 Extraction of .osm file for hobart_gccsa_2016_10000m_0 complete in 19.8 minutes.
 Extraction of .osm file for melb_gccsa_2016_10000m_0 complete in 20.4 minutes.
 Extraction of .osm file for mitchell_lga_2016_10000m_0 complete in 20.9 minutes.
 Extraction of .osm file for perth_gccsa_2016_10000m_0 complete in 21.4 minutes.
 Extraction of .osm file for syd_gccsa_2016_10000m_0 complete in 21.5 minutes.
 Extract

Note that the '21 cities' directory has since been re-set up as 'study_regions'. 

## Get networks and save as graphs (retain_all = True)

In [ ]:
# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/study_region/'
# output suffix
suffix = '_20181001'
count = 0
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if "10kmBuff" in file: 
           os.rename(file,file.replace('10kmBuff','_sua_2018_10000m_epsg4326').lower())
        if file.endswith("10000m_epsg4326.shp"): 
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = filename.replace('2016','2018')
           # Extract pedestrian network
           c = fiona.open(fullfile)   
           polygon = shape(next(iter(c))['geometry'])
           # Extract  complete non-private OSM network: "all (non-private) OSM streets and paths"
           W = ox.graph_from_polygon(polygon,  network_type= 'all', retain_all = True)
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_{studyregion}_all{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_{studyregion}_all{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)))
           W = ox.graph_from_polygon(polygon,  custom_filter= pedestrian, retain_all = True)
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_{studyregion}_pedestrian{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_{studyregion}_pedestrian{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)))  
           print('Saved graph object and shapefile for {} in {:.1f} minutes.'.format(studyregion,
                                                                                  (datetime.now() - subtime).total_seconds()/60))         
           count+=1          
                 
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))        

** Note that the below analyses are tangential and were performed prior to the above **

## Example of clean intersections for Melbourne

In [2]:
# We load up the saved network for nodes only; re-projection of nodes is cheap time-wise, but edges are expensive
G = ox.load_graphml('D:/ntnl_li_2018_template/data/study_region/melb/osm_10km_melb_gccsa_2016_10km_pedestrian_20181001.graphml')

In [ ]:
G_proj = ox.project_graph(G)

In [ ]:
intersections = ox.clean_intersections(G_proj, tolerance=15, dead_ends=False)

In [ ]:
points = np.array([point.xy for point in intersections])

## Test brisbane with retain_all = 'True'

Does this fix the island issue (Stradbroke networks missing) --- answer = yes.

In [7]:
# iterate of files within root or otherwise specified directory, noting all poly files

# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/study_region/bris'

# output suffix
suffix = '_20181011_retain_all'

count = 0
# Start timing the code
start_time = datetime.now()
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith("10000m_epsg4326.shp"): 
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = filename.replace('10000m_epsg4326','10km')
           # Extract pedestrian network
           c = fiona.open(fullfile)   
           polygon = shape(next(iter(c))['geometry'])
           # Extract  complete non-private OSM network: "all (non-private) OSM streets and paths"
           W = ox.graph_from_polygon(polygon,  network_type= 'all',retain_all ='True')
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_all{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_all{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix))) 
           W = ox.graph_from_polygon(polygon,  custom_filter= pedestrian,retain_all ='True')
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_pedestrian{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_pedestrian{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix))) 
           print('Saved graph object and shapefile for {} in {:.1f} minutes.'.format(studyregion,
                                                                                  (datetime.now() - subtime).total_seconds()/60))         
           count+=1          
                 
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))  

Saved graph object and shapefile for bris_gccsa_2016_10km in 24.8 minutes.
Elapsed time was 24.8 minutes


## Test brisbane with clean_periphery = 'False'
Does this fix the island issue (Stradbroke networks missing) --- No.

In [8]:
# iterate of files within root or otherwise specified directory, noting all poly files

# location of boundary files to iterate over
search_dir = 'D:/ntnl_li_2018_template/data/study_region/bris'

# output suffix
suffix = '_20181011_no_clean_periphery'

count = 0
# Start timing the code
start_time = datetime.now()
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.endswith("10000m_epsg4326.shp"): 
           subtime = datetime.now()
           fullfile = os.path.join(root,file)
           filename = os.path.splitext(file)[0]
           studyregion = filename.replace('10000m_epsg4326','10km')
           # Extract pedestrian network
           c = fiona.open(fullfile)   
           polygon = shape(next(iter(c))['geometry'])
           # Extract  complete non-private OSM network: "all (non-private) OSM streets and paths"
           W = ox.graph_from_polygon(polygon,  network_type= 'all',clean_periphery ='False')
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_all{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_all{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix))) 
           W = ox.graph_from_polygon(polygon,  custom_filter= pedestrian,clean_periphery ='False')
           ox.save_graphml(W, filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_pedestrian{suffix}.graphml'.format(studyregion = studyregion,
                                                                                                     suffix = suffix)), 
                           folder=None, gephi=False)
           ox.save_graph_shapefile(W, 
                                   filename=os.path.join(root,
                                                         'osm_10km_{studyregion}_pedestrian{suffix}'.format(studyregion = studyregion,
                                                                                                     suffix = suffix))) 
           print('Saved graph object and shapefile for {} in {:.1f} minutes.'.format(studyregion,
                                                                                  (datetime.now() - subtime).total_seconds()/60))         
           count+=1          
                 
print("Elapsed time was {:.1f} minutes".format((datetime.now() - start_time).total_seconds()/60.0))  

Saved graph object and shapefile for bris_gccsa_2016_10km in 25.2 minutes.
Elapsed time was 25.2 minutes
